In [8]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tensorflow
from tensorflow.python.framework import ops
import random
import json
import pickle

from numpy import unique

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings('ignore')

In [9]:
with open("/intents.json") as file:
    data = json.load(file)

words = []
labels = []
docs_x = []
docs_y = []

defaults = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        # делит на слова и знаки
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])
        
for default in data["defaults"]:
    defaults.append(default)
    
#print(words)

# массив всех известных слов - words

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

print("Number of words: ",len(words))

labels = sorted(labels)

print(labels)
print("Number of tags: ",len(labels))

# training - это  список массивов с 0 и 1, где 1 - это вхождение слова в большой список всех слов 
training = []
# для кождого массива - массив из 0 и 1, где 1 - напротив нужного класса (тэга)
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = numpy.array(training)
output = numpy.array(output)
   

def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        bag = bag_of_words(inp, words)
        if len(unique(bag)) <= 1:
            print(random.choice(defaults)) 
        else:
            results = model.predict([bag])
            results_index = numpy.argmax(results)
            tag = labels[results_index]

            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']

            print(random.choice(responses))

Number of words:  81
['doit', 'fine', 'goodbye', 'greeting', 'joke', 'mood', 'name', 'ok', 'thanks', 'weather']
Number of tags:  10


In [16]:
X_train, X_test, y_train, y_test = train_test_split(training, output, test_size=0.1, random_state=66)

In [19]:
# для подбора параметров
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)

# есть слово или нет в предложении - мои фичи 
# каждое дерево берет рандомное количество фич

# number of trees in random forest
n_estimators = [int(x) for x in numpy.linspace(start = 20, stop = 300, num = 100)]
# number of features at every split
max_features = ['auto', 'log2']

# max depth
max_depth = [int(x) for x in numpy.linspace(1, 100, num = 100)]
max_depth.append(None)
# create random grid
random_grid = {
 'n_estimators': n_estimators,
 'max_features': max_features,
 'max_depth': max_depth
 }
# Random search of parameters
rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=4, random_state=42, n_jobs = -1)
# Fit the model
rfc_random.fit(X_train, y_train)
# print results
print(rfc_random.best_params_)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:   14.1s
[Parallel(n_jobs=-1)]: Done 213 tasks      | elapsed:   30.7s
[Parallel(n_jobs=-1)]: Done 384 tasks      | elapsed:   51.3s
[Parallel(n_jobs=-1)]: Done 605 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 874 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  2.6min finished


{'n_estimators': 85, 'max_features': 'auto', 'max_depth': 73}


In [18]:
rfc = RandomForestClassifier(n_estimators=76, max_depth=33, max_features='auto')
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
rfc_cv_score = cross_val_score(rfc, training, output, cv=10)
print("Classification Report")
print(classification_report(y_test, rfc_predict))
print("Random Forest train score: ", rfc.score(X_train, y_train))
print("Random Forest test score:", rfc.score(X_test, y_test))
model = rfc

Classification Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      1.00      1.00         2
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         1
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       1.00      1.00      1.00         1

   micro avg       1.00      0.62      0.77         8
   macro avg       0.40      0.40      0.40         8
weighted avg       0.62      0.62      0.62         8
 samples avg       0.62      0.62      0.62         8

Random Forest train score:  0.9552238805970149
Random Forest test score: 0.625


In [ ]:
chat()

Start talking with the bot (type quit to stop)!
You: weather
Weather? I can't forecast it for tomorrow, but I like it today
You: hi
Hello there
You: how are you
I am good, thanks
